# Reinforcement Learning for Healthcare

In this notebook, we will be exploring reinforcement learning. We will begin by hypothesising the usefulness of reinforcement learning (RL) for healthcare; an application area in which it is seldom used. We will then go into the depths of RL, including the underlying statistical methodology of Markov decision processes (MDPs), and various RL approaches and paradigms. Finally, we hope to apply this knowledge to some examples to suggest how RL could be used in healthcare, especially in the context of multimorbidity trajectory modelling. 

## Intro & Motivation

Reinforcement Learning (RL) is a machine learning technique that has exploded in popularity in recent years, especially in areas with dynamic worlds such as real-time gaming and robotics. It is yet to gain popularity for use in healthcare, but there is some research emerging in this area, especially using offline RL. The primary goal of this project is to achieve a deep understanding of the methodology and mechanisms behind different types of RL (most of which are built on either a Q-learning or actor-critic based learning approach), as well as recent advances in deep RL. Through doing this, the goal is to then consider how RL can be used in the context of multimorbidity, by constructing an RL approach for modelling multimorbidity disease prediction over time ('multimorbidity trajectory modelling') using a multi-state model. Due to the methodological focus of this project, by studying the intricacies of different approaches in a thorough manner, the hope is that a deep understanding of RL will be gained. This will be useful in future research projects, whether that be applying RL to a specific healthcare problem, or even extending existing RL methodologies to advance the field. In summary, the objectives of this project (defined by C. Yau in https://github.com/cwcyau/hdruk-rl) are:
- To develop a deep understanding of reinforcement learning, including the differences between varying approaches including Q-learning, actor-critic, and deep extensions.
- To understand how to model multimorbidity trajectories using multi-state models.
- To implement an RL approach to learn multi-state models for modelling multimorbidity trajectories. 

In this notebook, I will explain some of the key concepts, providing examples. Some of this content is inspired by DeepLizard's course on RL, which provides youtube videos on the key aspects of RL: https://youtube.com/playlist?list=PLZbbT5o_s2xoWNVdDudn51XM8lOuZ_Njv

## Markov Decision Processes

### What are they?

Markov decision processes (MDPs) form the basis of reinforcement learning problems. Let's focus on the main components of an MDP in the context of RL:
- <b>Agent</b>: This is the active decision maker in the problem. This could be anything: a person, an animal, a game character, a self-driving car, etc. This <b>agent</b> interacts with its <b>environment</b>.
- <b>Environment</b>: The environment is the 'world'; where exciting and scary things might happen, and the <b>agent</b> wants to be prepared to handle these and act accordingly. 
- <b>State</b>: The <b>state</b>, $S_t$, is the current situation. This is composed of the environment's status at each timestep $t$, as well as the agent's position within the environment. With each timestep, the agent makes a choice about what to do (performs an action) given its environment, and will transition to a new state.
- <b>Action</b>: The action is the choice that the agent makes at each timestep, given its environment and current state. This leads an agent from one state to another. The agent wants to make a good choice, and so each action has a corresponding reward - or set of possible rewards in a non-deterministic world. 
- <b>Reward</b>: A reward is recieved at each timestep, after the agent transitions from one state $S_t$ to another, $S_t+1$. The rewards are cumulative, so the aim of the agent is to maximise the cumulative rewards over time, not just the individual reward at each timestep. It 'looks ahead' to estimate this, but more weight is given to rewards that are in the near future rather than far away by discounting estimated rewards far away - this is called the _discounted reward_, and the discount is always less than 1). Rewards can also be negative.
- <b>Transition Probability</b>: This is the probability that at time $t$, from state $S_t$, the agent will transition to state $S_t+1$ given it performs action $a$. This is 1 if the actions are deterministic (i.e. each action has only one outcome state). 

I'll now give a 'fun' health-related example that vaguely mimics a simplified version of real-life to illustrate this. At each time _t_ we have:
- Agent: a person
- Environment: the earth/their life 
- State: the set of diseases/comorbidities the person has 
- Actions: lifestyle choices or medications that affect the likelihood of the agent developing or losing comorbidities
- Reward: the reward of taking an action and ending in a different state. For example, the reward of smoking might be very negative since it will increase the likelihood of developing cancer or high blood presure, and the reward of exercise may be positive but small since it results in no change to comorbidities. A simple reward function can be determined as the number of comorbidities present in a given state, for example.

The ambition of the agent in this example is to battle diseases and remain as healthy as possible with minimal comorbidities, hence extending their lifespan; this will give the highest cumulative reward. 

### Formal representation and definitions

It should help with clarity to formalise the definitions above in mathematical notation. In this way, an MDP can be defined as follows:
- $S$ is the set of states, $A$ is the set of actions, $R$ is the set of rewards.
- We have $n$ timesteps $t$, where $t = 0,1,2,...,n-1$.
- The states at each $t$ is $S_t$, where $S_t \in S$.
- The current state at each $t$ is $s$, where $s \in S_t \in S$.
- At each $t$, the agent $ag$ performs one action $a$, where $A_t$ is the set of possible actions, and $a \in A_t \in A$.
- The action taken from a given state can be defined as a state-action pair $(s, a)$.
- The $ag$ recieves a reward $r$ after taking action $a$ from state $s$. This happens with a given transition probability, and $ag$ moves to the next state, $s'$.
- The transition probability from $(s, a)$ to $s'$ can be defined as $p(s')$, as it is simply the probability of ending up in that state. This also means that $p(r = p(s'))$, which is chosen from a selection of possible next states, $S_{t+1}$.

In general, $s' \in S_{t+1} \in S$ and $r \in R_{t+1} \in R$. In a deterministic world, $s = S_{t}$ and $r = R_{t+1}$, as following each $(s, a)$ results in only one possible outcome state $s'$ with one possible corresponding reward $r$.

At each timestep, we reset the current state $s = s'$. The set of possible actions, $A_t$ is likely also updated.

Hopefully as we continue with examples, this will all become clearer. 

### Code example

We can illustrate building the MDP described above in a simple code example. 
We begin with an individual with 0 comorbidities. We have 10 timesteps, and 3 comorbidities (obesity; heart disease; diabetes) that can develop or be cured. We have 3 possible actions that can be taken from each state, these are treatments (statins; stop smoking; exercise). In this example, the transition probabilities between each state are randomly generated. We print out each state and the transititon probabilities for each time t.

In [6]:
import numpy as np 
import random as rand
#start off with dictionary of comorbidities being all 0s 
#then update this as we move through the mdp/policy 

comorbid_dict = {
    'diabetes':0,
    'heart_disease':0,
    'obesity':0
}

# finite number of states, encapsulating the possible comorbid dict values
states = [{'diabetes':0,'heart_disease':0,'obesity':0},
          {'diabetes':1,'heart_disease':0,'obesity':0},
          {'diabetes':1,'heart_disease':1,'obesity':0},
          {'diabetes':1,'heart_disease':0,'obesity':1},
          {'diabetes':1,'heart_disease':1,'obesity':1},
          {'diabetes':0,'heart_disease':1,'obesity':0},
          {'diabetes':0,'heart_disease':1,'obesity':1},
          {'diabetes':0,'heart_disease':0,'obesity':1}]

#actions are interventions such as drugs or lifestyle changes 
actions = ['statin', 'stop_smoking','exercise']

#at each state, for each action, we transition to another state with a random probability 

max_t = 10
t=0

max_prob = 0.0

while t < 10:
    for s in states:
        print('start state:',s)
        for a in actions:
            print('action taken:', a )
            i = 0
            #generate random transition probabilities for each state, action pair 
            random_probs = np.random.random(len(states))
            #make sure they add up to 1 
            random_probs /= random_probs.sum() 
            for s_1 in states:
                state_action = frozenset(sorted(s.items())),a
                curr_prob = random_probs[i]
                outcome_transition = frozenset(sorted(s_1.items())), curr_prob
                transition = {state_action:outcome_transition}
                print('next state', s_1)
                print('transition probability', random_probs[i])
                i+=1
            i=0
        t+=1

start state: {'diabetes': 0, 'heart_disease': 0, 'obesity': 0}
action taken: statin
next state {'diabetes': 0, 'heart_disease': 0, 'obesity': 0}
transition probability 0.1363203799910454
next state {'diabetes': 1, 'heart_disease': 0, 'obesity': 0}
transition probability 0.1698211129824225
next state {'diabetes': 1, 'heart_disease': 1, 'obesity': 0}
transition probability 0.12381625622735117
next state {'diabetes': 1, 'heart_disease': 0, 'obesity': 1}
transition probability 0.15270476522685475
next state {'diabetes': 1, 'heart_disease': 1, 'obesity': 1}
transition probability 0.14299318706438185
next state {'diabetes': 0, 'heart_disease': 1, 'obesity': 0}
transition probability 0.057047637676591184
next state {'diabetes': 0, 'heart_disease': 1, 'obesity': 1}
transition probability 0.14253818293011317
next state {'diabetes': 0, 'heart_disease': 0, 'obesity': 1}
transition probability 0.07475847790124002
action taken: stop_smoking
next state {'diabetes': 0, 'heart_disease': 0, 'obesity': 

Now that this MDP is built, we can solve this to find the optimal <b>policy</b>. A policy is defined as:
- <b>Policy</b>: The policy is a function that tells an agent, from a given state, the probabilities of selecting each possible action. In a deterministic policy, it is a function that maps from a given state to only one action. The optimal policy is the policy with the highest expected cumulative reward (we will come to how this is calculated in a moment).

Formally:
- At timestep $t$, the policy can be defined as $Pol$, so the probability of selecting action $a$ from state $s$ can be written as $Pol(a|s)$.
- Therefore, for each state $s$, $Pol$ is a probability distribution over $a \in A_t$, which can be written as $Pol = a \in A_t(s)$.

What reinforcement learning does is find the optimal policy by exploring and exploiting the environment. It _learns_ this policy. In the upcoming sections, we will consider two of the main RL approaches: Q-learning and actor-critic learning, to illustrate this. 

But first, we must consider how to calculate the _expected rewards_ mentioned earlier. This basically means, how can we estimate how good it is to be in each state? Therefore estimating, how good is it to take a specific action from a given state? We can do this with help from the <b>value function</b>.


### Value Functions

Value functions are used to estimate the expective cumulative reward of a particular policy. This differs from the <b>reward function</b> in that this is the reward an agent actually recieves at each state, rather than the expected cumulative reward. 

Value functions are therefore defined with respect to policies, and there are two different types of value functions: 
1. <b>State value functions</b>: These estimate how 'good' any given state is at a particular time, when an agent is following a specific policy, denoted as $v_{Pol}$, which can be thought of as the 'value' of a state; this is different to the reward of the state in that it is the sum of the cumulative rewards up until this state, and the estimate of the rewards from this state (rather than just the reward of that single state). This can be written as $v_{Pol}(S_t)$. 


2. <b>Action value functions</b>: These estimate how 'good' a given action is, from a given state at a particular time, following a given policy - thought of as the 'value' of taking a given action. It is denoted as $q_{Pol}$ and can therefore be written as $q_{Pol}(S_t,A_t)$. This is referred to as the <b>Q-function</b>, where Q stands for _quality_ 😎. 

This is where we get to the exciting part about reinforcement learning, actually solving the MDPs to generate optimal policies...

### Optimal Policies 

An optimal policy can simply be thought of as the policy that gives the greatest (or equally great) expected cumulative reward over all states in comparison to all alternative policies. Basically, it's the best path to take. 

Given this, it should come as no surprise that _optimal policies have corresponding optimal state value functions and action value functions_.

Perhaps unsurprisingly, the optimal state value function is that with the highest expected value achievable by any policy for each state, for all states. This can be denoted as $v^{*}_{Pol}(s)$ for all $S$.

Likewise, the optimal action value function (Q-function) is that with the largest expected value achievable by any policy for each possible state-action pair, for all states and all actions. This can be denoted as $q^{*}(s,a)$ for all $S$ and all $A(s)$.

- <b>q*</b> must satisfy the _Bellman optimality equation_.
    
Sounds cool, right? It is. And it's important to remember as this is a key aspect of the Q-function. It states:

For any state, action pair $(s,a)$ at timestep $t$, the Q-value is the expected reward for taking $a$ from $s$ (denoted as $r$), plus the _maximum_ expected discounted reward from _any_ possible next state, action pair $(s', a')$, by following the optimal policy from thereon. So then $(s')$ is the best possible next state, from which the best possible next action $(a')$, can be taken in the following timestep... etc. This forms the optimal Q-function, $q^{*}$.  In maths, we can say:

$q^{*}(s,a) = E[r + max q^{*}(s', a')]$.
    
This might sound confusing at first but it's actually quite logical, and it should help when we explore how this is actually used. We use this equation to find $q^{*}$ (i.e. the best q-function). We then use RL to find the best $a$ from each $s$, following this policy, and hence determine the optmal policy. 
    
This is where - finally - we get to the exciting stuff: how to do this using Q-learning! 

## Q-Learning

In this section we will explore Q learning, which belongs to the family of RL approaches called value-based methods.

### What is Q-Learning?

Q-learning is a reinforcement learning method of solving an MDP, which learns the optimal policy. This means, that the policy learned has the highest expected cumulative reward over all successive steps compared to alternative policies. 

Q-learning does this by learning the optimal Q-values for each state, action pair. This is $q*(s,a)$ and is calculated using the Q-function; which takes a state and and action and returns the expected discounted reward when following a given policy.

Q-learning works using the concept of <b>Value Iteration</b>.

### Value Iteration

As we know, each state, action pair has a Q-value, $q$. 
Q-learning simply iteratively updates this value using the Bellman equation, by moving through the possible states and actions. It does this until the Q-function converges to the optimal Q-function, q*. This is value iteration. Here's a simple description of value iteration, in steps:

In steps:
- The agent begins with no idea about what future actions and states might exist - it knows only its current environment. It therefore also has no idea how good any action and its corresponding rewards will be. Because of this, all of the Q-values for each state, action pair can be initialised to 0. These are updated using value iteration, as the agent moves through its environment, and stored in a Q-table, which is a grid of values, with one entry for each action, state pair (ignoring time). 
- The agent makes a choice of any action from its start state. It then lands in another state, and recieves a reward. The Q-value is updated for the first state, action pair. It then moves from this state, into another, by choosing another action, and recieves another reward. This will continue until the timesteps limit is reached (_max steps_, which can be configured), or the 'game' is ended for another reason (i.e. in the 'game of life', the individual dies). 
- The agent will start again, and complete the same process, updating the Q-values as it moves through the world. In later simulations of this (the number of 'games played' can also be configured), the agent can have a look in the table, and see what a good next action to take might be.  
    - BUT the danger of this is that it'll just choose the same action over and over again, and not explore enough alternative options (i.e. get stuck in a local maxima by <b>exploiting</b> the environment). 
    - HOWEVER if it just keeps <b>exploring</b> forever, and ignores the Q-values found already, it will never converge. 
- So, in order to decide which paths to take, Q-learning utilises the <b>explore/exploit tradeoff</b> - which can be configured.


In [7]:
#Give an example of this here 

### Q-learning explore/exploit strategy - Epsilon Greedy

So, how do we configure the explore/exploit tradeoff? 

We can define an <b>exploration rate</b>, $\epsilon$. This is the probability of choosing to explore rather than exploit. This is initially set to 1, to ensure that the agent starts off by exploring. 

$\epsilon$ is decreased each time the game is started. So, as the agent learns more about the environment, the chance of exploiting increases and the agent becomes "_greedy_". Remember, exploring is randomly choosing an action, exploiting is choosing the action with the highest associated Q-value from the current state. However, when do we update this Q-value, if the state, action pair is revisited (e.g. at a later timestep) and the cumulative discounted reward therefore changes? 

This is done by using the <b>learning rate</b>, $Lr$: Between 0 and 1, this is basically how quickly an agent decides to update a Q-value for a given state, action pair. If set to 1, the old value will be completely replaced by the new value. 0 leaves the value as-is. Anything in between however, considers both the old and the new Q-values for that position - a higher $Lr$ results in more weight given to the new value. It is simply the weighted sum of the new _learned value_,  i.e. the cumulative discounted reward found for that state, action pair; and the old Q-value, following the calculation:

$q'(s,a) = (1-Lr)q(s,a) + Lr(r + max q(s',a'))$

Where $(r + max q(s',a'))$ is the new _learned value_, and $q(s,a)$ is the old Q-value.

This $q'(s,a)$ then replaces the previous Q-value.

This process continues, with multiple 'game plays', until the Q-function converges, and the optimal policy is found. 

### Code implementation 

Building on the simple MDP code we wrote earlier, we can implement this strategy to find the optimal q-function, q*, and optimal policy. 

For this, we will want to construct a q-table, to keep track of the q-values during value iteration, and set the parameters for the exploration and learning rate to ensure the explore/exploit tradeoff is considered. 

In [2]:
## code


Now that has hopefully made clear the basics of Q-learning and how a simple example might be implemented, we will shift focus to another key reinforcement learning paradigm: policy-gradient methods. These are appealing since rather than generating deterministic policy (like Q-learning does, using a value-based approach), they return stochastic policies, which are distributions over actions - this makes more sense for a lot of RL problems. However, there are issues with the basic policy-gradient approach, which we will explain - this has lead to the development of actor-critic learning techniques. We will go into these in detail, in a similar way as we did for Q-learning, to ensure understanding of this approach in the next notebook!